In [1]:
import numpy as np, scipy, matplotlib.pyplot as plt, IPython.display as ipd, sklearn
import librosa, librosa.display
import os
import warnings
import tqdm
import json
import pickle
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
SOUND_FOLDER_training = "./Audio_Dataset/training_data/"
SOUND_FOLDER_testing = "./Audio_Dataset/testing_data/"
FEAT_FOLDER = "./Features"

In [3]:
# Utilities

def sigmoid_function(x):
    return 1.0/(1.0+np.exp(-x))


---
# **FEATURE EXTRACTION**




In [4]:
# Feature Extraction

def extract_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=22)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc, mfcc_mean


In [5]:
path_training = os.path.join(SOUND_FOLDER_training)
path_testing = os.path.join(SOUND_FOLDER_testing)

In [6]:
# Retrieve list of files
list_of_files_training = []
list_of_files_testing = []

# dirs=directories
for (root, dirs, file) in os.walk(path_training):
    for f in file:
        if '.wav' in f:
            list_of_files_training.append(root+"/"+f)

for (root, dirs, file) in os.walk(path_testing):
    for f in file:
        if '.wav' in f:
            list_of_files_testing.append(root+"/"+f)

print(list_of_files_training)
print(len(list_of_files_training)) # 160 files = 80 disco + 80 hiphop
print(list_of_files_testing)
print(len(list_of_files_testing)) # 40 files = 20 disco + 20 hiphop

['./Audio_Dataset/training_data/disco/disco.00052.wav', './Audio_Dataset/training_data/disco/disco.00046.wav', './Audio_Dataset/training_data/disco/disco.00047.wav', './Audio_Dataset/training_data/disco/disco.00053.wav', './Audio_Dataset/training_data/disco/disco.00045.wav', './Audio_Dataset/training_data/disco/disco.00051.wav', './Audio_Dataset/training_data/disco/disco.00079.wav', './Audio_Dataset/training_data/disco/disco.00078.wav', './Audio_Dataset/training_data/disco/disco.00050.wav', './Audio_Dataset/training_data/disco/disco.00044.wav', './Audio_Dataset/training_data/disco/disco.00068.wav', './Audio_Dataset/training_data/disco/disco.00040.wav', './Audio_Dataset/training_data/disco/disco.00054.wav', './Audio_Dataset/training_data/disco/disco.00055.wav', './Audio_Dataset/training_data/disco/disco.00041.wav', './Audio_Dataset/training_data/disco/disco.00069.wav', './Audio_Dataset/training_data/disco/disco.00057.wav', './Audio_Dataset/training_data/disco/disco.00043.wav', './Audio_

In [7]:
# Define the Dataframes of features
feat_df_training = pd.DataFrame(columns=['file_name'])
feat_df_testing = pd.DataFrame(columns=['file_name'])

In [8]:
# -- FEATURE EXTRACTION --

# for TRAINING data

for fname in tqdm.tqdm(list_of_files_training):
  signal, sr = librosa.load(path=fname)

  db_new_row = {'path': fname}
  db_new_row['file_name'] = os.path.basename(fname).split('/')[-1]
  
  # MFCC
  feat, feat_mean = extract_mfcc(y=signal, sr=sr)
  for i, val in enumerate(feat_mean):
    db_new_row["MFCC_mean_"+str(i)] = val

  s = pd.Series(db_new_row)
  feat_df_training = pd.concat([feat_df_training, s.to_frame().T], ignore_index=True)

100%|██████████| 160/160 [00:09<00:00, 16.00it/s]


In [9]:
# -- FEATURE EXTRACTION --

# for TESTING data

for fname in tqdm.tqdm(list_of_files_testing):
  signal, sr = librosa.load(path=fname)

  db_new_row = {'path': fname}
  db_new_row['file_name'] = os.path.basename(fname).split('/')[-1]
  
  # MFCC
  feat, feat_mean = extract_mfcc(y=signal, sr=sr)
  for i, val in enumerate(feat_mean):
    db_new_row["MFCC_mean_"+str(i)] = val

  s = pd.Series(db_new_row)
  feat_df_testing = pd.concat([feat_df_testing, s.to_frame().T], ignore_index=True)

100%|██████████| 40/40 [00:01<00:00, 21.82it/s]


In [10]:
feat_df_training

,file_name,path,MFCC_mean_0,MFCC_mean_1,MFCC_mean_2,MFCC_mean_3,MFCC_mean_4,MFCC_mean_5,MFCC_mean_6,MFCC_mean_7,...,MFCC_mean_12,MFCC_mean_13,MFCC_mean_14,MFCC_mean_15,MFCC_mean_16,MFCC_mean_17,MFCC_mean_18,MFCC_mean_19,MFCC_mean_20,MFCC_mean_21
0,disco.00052.wav,./Audio_Dataset/training_data/disco/disco.0005...,-70.546455,90.50209,-48.061337,36.668949,-17.682131,21.58345,-30.1917,24.033106,...,-12.047788,6.267945,-11.548881,4.907653,-7.875762,-0.645613,-4.357143,0.19585,-7.518898,2.101671
1,disco.00046.wav,./Audio_Dataset/training_data/disco/disco.0004...,-125.732597,101.787453,-17.478825,33.670853,-15.239169,23.698208,-8.662551,18.543148,...,-14.305062,3.032468,-7.08803,1.622791,-3.898687,5.380833,0.963895,2.688429,-4.63821,-1.965669
2,disco.00047.wav,./Audio_Dataset/training_data/disco/disco.0004...,-280.854309,120.812538,-37.629761,28.712122,-10.775235,-6.40532,-13.794428,-4.07455,...,-6.790002,-6.412713,-1.754004,-4.118971,3.408171,4.254298,0.9962,-1.16373,-1.29808,-5.560733
3,disco.00053.wav,./Audio_Dataset/training_data/disco/disco.0005...,-108.564842,69.971283,14.885138,45.513443,-5.374344,41.795094,-13.517867,32.566601,...,-9.027057,7.001934,-4.725863,2.343729,0.201238,2.34515,-0.78258,-0.591641,-3.213276,0.005196
4,disco.00045.wav,./Audio_Dataset/training_data/disco/disco.0004...,-81.305023,106.479553,-9.540607,17.966782,3.010284,8.300962,3.889344,4.250727,...,-1.163623,-1.877596,-0.224992,-1.04435,-3.28482,-1.87133,-0.713636,-0.551896,1.247898,1.869075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,hiphop.00010.wav,./Audio_Dataset/training_data/hiphop/hiphop.00...,-67.462418,94.848267,-38.849628,37.633415,-12.246916,36.890293,-12.640728,18.295763,...,-8.736391,4.5039,-7.594248,0.212765,-4.881601,4.845163,-1.99476,-0.738198,-4.111737,-1.777408
156,hiphop.00006.wav,./Audio_Dataset/training_data/hiphop/hiphop.00...,-59.565926,63.929668,3.487175,45.235748,-1.152862,28.24374,-13.170861,23.591469,...,-5.800775,8.173893,-6.858704,6.735177,-4.482601,0.953162,-7.124492,-1.256965,-3.602875,1.350129
157,hiphop.00012.wav,./Audio_Dataset/training_data/hiphop/hiphop.00...,-67.06469,112.404198,-23.642937,44.273914,-18.864435,23.873178,-13.402471,20.146372,...,-9.388905,11.289869,-8.865541,7.08106,0.446763,9.378616,-0.892912,-0.576769,2.853826,1.891011
158,hiphop.00013.wav,./Audio_Dataset/training_data/hiphop/hiphop.00...,-129.323669,141.712906,-42.477188,34.013855,-6.975853,8.702186,-11.367335,15.941672,...,-5.472686,5.869145,-3.156407,-1.692448,-8.804316,-4.851142,-9.280404,2.128211,9.270855,15.514466


In [11]:
feat_df_testing

,file_name,path,MFCC_mean_0,MFCC_mean_1,MFCC_mean_2,MFCC_mean_3,MFCC_mean_4,MFCC_mean_5,MFCC_mean_6,MFCC_mean_7,...,MFCC_mean_12,MFCC_mean_13,MFCC_mean_14,MFCC_mean_15,MFCC_mean_16,MFCC_mean_17,MFCC_mean_18,MFCC_mean_19,MFCC_mean_20,MFCC_mean_21
0,disco.00091.wav,./Audio_Dataset/testing_data/disco/disco.00091...,-116.88237,98.471169,-18.994415,44.840286,3.802606,28.490913,-1.105584,14.694503,...,-3.911044,7.904995,-3.217057,0.130555,-7.842556,-1.726864,-2.423464,-0.718608,-2.856908,-4.202101
1,disco.00085.wav,./Audio_Dataset/testing_data/disco/disco.00085...,-166.884155,67.677948,11.208783,37.441818,-10.936877,27.395809,-18.942213,27.296139,...,-6.046257,12.007288,-6.077664,7.27495,-4.232254,5.127122,-4.525802,3.458847,-2.264731,-0.596862
2,disco.00084.wav,./Audio_Dataset/testing_data/disco/disco.00084...,-197.216492,83.347702,-38.580997,43.67588,-7.302443,25.634623,-8.659459,13.366769,...,-2.39933,12.016626,1.38305,6.762504,-3.372065,-1.6579,0.067833,1.405344,-1.602576,-2.745158
3,disco.00090.wav,./Audio_Dataset/testing_data/disco/disco.00090...,-125.691124,88.540619,-16.764915,42.78476,-9.905494,29.165888,-19.357843,20.462629,...,-7.757985,7.044875,-8.517818,3.274963,-6.478028,5.800038,-3.495919,-0.057122,-6.214284,0.060961
4,disco.00086.wav,./Audio_Dataset/testing_data/disco/disco.00086...,-95.783737,122.072479,-29.341267,50.330864,-11.952024,28.39348,-7.055488,11.860459,...,-11.656873,2.684297,-5.794095,0.648301,-7.260707,1.747831,-6.149038,-3.747774,-1.353693,-1.251166
5,disco.00092.wav,./Audio_Dataset/testing_data/disco/disco.00092...,-123.838387,89.855423,-1.846599,51.166451,-16.249851,28.27051,-12.485133,19.029924,...,-8.755854,8.2524,-8.232474,5.514292,-8.406771,4.474397,0.313075,1.827121,-9.063951,-4.927888
6,disco.00093.wav,./Audio_Dataset/testing_data/disco/disco.00093...,-86.060478,87.539619,-20.164946,53.994804,-1.049911,19.554424,-4.069276,21.033278,...,-15.912197,3.974922,-13.915902,2.380633,-7.735189,3.442756,-4.146654,5.921882,-3.141375,2.296835
7,disco.00087.wav,./Audio_Dataset/testing_data/disco/disco.00087...,-100.280724,109.986755,-26.644407,36.837772,-10.409534,28.786537,-14.84601,21.287558,...,-13.790873,3.991437,-10.454129,2.846874,-4.31649,3.170058,-3.742944,3.561872,-2.476716,-0.627575
8,disco.00083.wav,./Audio_Dataset/testing_data/disco/disco.00083...,-167.48877,112.504593,-4.578629,37.091393,-8.612248,27.814657,-14.704628,18.51487,...,-11.572749,6.742621,-9.19417,4.897453,-9.71392,1.520472,-5.570963,-2.067122,-2.047593,-1.452411
9,disco.00097.wav,./Audio_Dataset/testing_data/disco/disco.00097...,-116.932053,75.749084,-4.22155,37.290447,-5.326528,31.376789,-12.156291,22.918867,...,-13.21757,8.250454,-15.219112,9.844795,-6.203883,8.005424,-8.028849,4.383463,0.95365,1.551707


---
# **SAVE FEATURE FILES**


In [12]:
path = os.path.join(FEAT_FOLDER)
print("save features in " + path)

isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

print("save features to -> "+path + "feature_training.pkl")
with open(os.path.join(path, "feature_training.pkl"), 'wb') as fout:
   pickle.dump(feat_df_training, fout, protocol=pickle.HIGHEST_PROTOCOL)

print("save features to -> "+path + "feature_testing.pkl")
with open(os.path.join(path, "feature_testing.pkl"), 'wb') as fout:
   pickle.dump(feat_df_testing, fout, protocol=pickle.HIGHEST_PROTOCOL)

save features in ./Features
save features to -> ./Featuresfeature_training.pkl
save features to -> ./Featuresfeature_testing.pkl
